In [1]:
from __future__ import print_function
%matplotlib notebook
#%matplotlib inline
import numpy as np
import math
import random

import torch
import torch.nn.parallel

In [2]:
import sys

from sVAEs_module import TrainSVAE

sys.path.append('code/TreeGAN/dataloader')
from dataset_benchmark import BenchmarkDataset, BenchmarkDatasetOnTheFly

In [3]:
SEED = 0 
random.seed(SEED) 
np.random.seed(SEED) 
torch.manual_seed(SEED) 

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
# device = torch.device('cpu')
print(device)

cuda:0


/home/shidi/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:104: UserWarning: 
GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70 sm_75.
If you want to use the GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [5]:
############################## Data loader ####################################
print('Loading data.........')
# Root directory for dataset
dataroot = '../../data/datasetTreeGAN/shapenetcore_partanno_segmentation_benchmark_v0/'

category = 'chair'
# data = BenchmarkDataset(dataroot, npoints=2048, uniform=False, classification=False, class_choice=category, device=device)
data = BenchmarkDatasetOnTheFly(dataroot, npoints=2048, uniform=False, classification=False, class_choice=category, device=device)

split = 0.9
total_num = data.__len__()
train_num = math.floor(total_num*split)
test_num = total_num - train_num
train_set, test_set = torch.utils.data.random_split(data, [train_num, test_num])

batch_size = 30
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=0, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=0, drop_last=True)

print(total_num)

Loading data.........
{'chair': 0}
3746


In [6]:
sp_reg_terms = {"regularizer_type": ['bernoulli_regularizer', 'parsimony_regularizer', 'overlapping_regularizer'],
                 "bernoulli_regularizer_weight": 1,
                 "entropy_bernoulli_regularizer_weight": 0,
                 "parsimony_regularizer_weight": 1e-3,
                 "sparsity_regularizer_weight": 0,
                 "overlapping_regularizer_weight": 1e-2,
                 "minimum_number_of_primitives": 4,
                 "maximum_number_of_primitives": 10,
                 "w1": 0.005,
                 "w2": 0.005}
# sp_reg_terms = {"regularizer_type": ['bernoulli_regularizer', 'sparsity_regularizer', 'overlapping_regularizer'],
#                  "bernoulli_regularizer_weight": 1,
#                  "entropy_bernoulli_regularizer_weight": 0,
#                  "parsimony_regularizer_weight": 0,
#                  "sparsity_regularizer_weight": 1,
#                  "overlapping_regularizer_weight": 1e-2,
#                  "minimum_number_of_primitives": 6,
#                  "maximum_number_of_primitives": 10,
#                  "w1": 0.005,
#                  "w2": 0.005}
# sp_reg_terms = {"regularizer_type": ['bernoulli_regularizer', 'parsimony_regularizer'],
#                  "bernoulli_regularizer_weight": 1,
#                  "entropy_bernoulli_regularizer_weight": 0,
#                  "parsimony_regularizer_weight": 1e-3,
#                  "sparsity_regularizer_weight": 0,
#                  "overlapping_regularizer_weight": 0,
#                  "minimum_number_of_primitives": 4,
#                  "maximum_number_of_primitives": 10,
#                  "w1": 0.005,
#                  "w2": 0.005}

In [7]:
vae = TrainSVAE(beta=1e-4, batch_size=batch_size, epoch=2,
                train_loader=train_loader, test_loader=test_loader,
                category = category, device=device, 
                sp_reg_terms=sp_reg_terms, use_realNVP=True)

path:  sVAE_chair_beta00001_nparts8_nlatents256_regBern1_regParsimony0001_regSparsity0_regOverlap001_min4_max10_realNVP


In [8]:
vae.run(threshold_epoch=2000)
# vae.load()

# vae.reconstruction()
# vae.vis()
# vae.vis_generated(distribution='S_Gaussian')
# flow = vae.realNVP()
# vae.load_realNVP()

RuntimeError: CUDA error: no kernel image is available for execution on the device

In [ ]:
# vae.estimate_latent()

In [ ]:
# vae.vis_generated(distribution='M_Gaussian')
# vae.reconstruction()

In [ ]:
# vae.generate('M_Gaussian')
# print('Mixed Gaussian, MMD-CD: ', vae.mmd_generate(device=device))
# print('Mixed Gaussian, COV-CD: ', vae.cov_generate(device=device))
# # # # print('Mixed Gaussian, COV-EMD: ', vae.cov_generate(distance='EMD', device=device))
# print('Mixed Gaussian, JSD: ', vae.jsd_generate())
# vae.generate('M_Gaussian', parts=True)
# print('Mixed Gaussian, MMD-CD per-Parts: ', vae.mmd_generate(per_parts=True, sp=vae.get_seg, device=device))
# # # print('Mixed Gaussian, COV-CD per-Parts: ', vae.cov_generate(per_parts=True, sp=vae.get_seg, device=device))
# # # print('Mixed Gaussian, COV-EMD per-Parts: ', vae.cov_generate(distance='EMD', per_parts=True, device=device))
# # # print('Mixed Gaussian, JSD per-Parts: ', vae.jsd_generate(per_parts=True, sp=vae.get_seg, device=device))

# vae.generate('S_Gaussian')
# print('Standard Gaussian, MMD-CD: ', vae.mmd_generate(device=device))
# print('Standard Gaussian, COV-CD: ', vae.cov_generate(device=device))
# # print('Standard Gaussian, COV-EMD: ', vae.cov_generate(distance='EMD', device=device))
# print('Standard Gaussian, JSD: ', vae.jsd_generate())
# vae.generate('S_Gaussian', parts=True)
# print('Standard Gaussian, MMD-CD per-Parts: ', vae.mmd_generate(per_parts=True, sp=vae.get_seg, device=device))
# # # print('Standard Gaussian, COV-CD per-Parts: ', vae.cov_generate(per_parts=True, sp=vae.get_seg, device=device))
# # # print('Standard Gaussian, COV-EMD per-Parts: ', vae.cov_generate(distance='EMD', per_parts=True, device=device))
# # # print('Standard Gaussian, JSD per-Parts: ', vae.jsd_generate(per_parts=True, sp=vae.get_seg,))

In [ ]:
# vae.device = torch.device('cpu')
# vae.latent_stat()